# Model Registry

In [3]:
import mlflow
from mlflow import MlflowClient
import os
from dotenv import load_dotenv
load_dotenv()

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(os.getenv("EXPERIMENT_NAME"))

<Experiment: artifact_location='s3://mlops-bucket.store.nus.cloud/48', creation_time=1760285277317, experiment_id='48', last_update_time=1760285277317, lifecycle_stage='active', name='fraud_detection_dev1', tags={'mlflow.experimentKind': 'custom_model_development'}>

## Register a model to Model Registry

In [4]:
"""
mlflow.<model_flavor>.log_model(registered_model_name=<model_name>) # register the model while logging it to the tracking server.
mlflow.register_model(<model_uri>, <model_name>) # register the model after logging it to the tracking server. Note that you'll have to log the model before running this command to get a model URI.
"""

"\nmlflow.<model_flavor>.log_model(registered_model_name=<model_name>) # register the model while logging it to the tracking server.\nmlflow.register_model(<model_uri>, <model_name>) # register the model after logging it to the tracking server. Note that you'll have to log the model before running this command to get a model URI.\n"

## Load a Registered Model - To perform inference on a registered model version

In [5]:
# Use model ID
model_id = 'm-3e2c49f2b3fa4afd82a0e4f8dc05eceb'
logged_model = mlflow.get_logged_model(model_id)

print(logged_model, logged_model.metrics, logged_model.params)
print(logged_model.version)

LoggedModel(artifact_location='s3://mlops-bucket.store.nus.cloud/1/models/m-3e2c49f2b3fa4afd82a0e4f8dc05eceb/artifacts', creation_timestamp=1758473672239, experiment_id='1', last_updated_timestamp=1758473674998, model_id='m-3e2c49f2b3fa4afd82a0e4f8dc05eceb', model_type='', model_uri='models:/m-3e2c49f2b3fa4afd82a0e4f8dc05eceb', name='logreg_model', source_run_id='0213accbaff94aeaa6d30a600895b6e2', status=<LoggedModelStatus.READY: 'READY'>, status_message='') [<Metric: dataset_digest=None, dataset_name=None, key='train_accuracy', model_id='m-3e2c49f2b3fa4afd82a0e4f8dc05eceb', run_id='0213accbaff94aeaa6d30a600895b6e2', step=0, timestamp=1758473675713, value=0.9555273189326556>, <Metric: dataset_digest=None, dataset_name=None, key='train_precision', model_id='m-3e2c49f2b3fa4afd82a0e4f8dc05eceb', run_id='0213accbaff94aeaa6d30a600895b6e2', step=0, timestamp=1758473675851, value=0.986449864498645>, <Metric: dataset_digest=None, dataset_name=None, key='train_recall', model_id='m-3e2c49f2b3fa4

AttributeError: 'LoggedModel' object has no attribute 'version'

In [ ]:
# Use run ID
# model = mlflow.sklearn.load_model(f"runs:/{mlflow_run_id}/{run_relative_path_to_model}")


In [ ]:
# Use registered model name, and version
model_uri = "models:/fraud-detection-model/3" #"models:/{registered_model_name}/{model_version}"
model = mlflow.sklearn.load_model(model_uri)

print(model)

/Users/carlychinsekyi/miniconda3/envs/mlops/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LogisticRegression(max_iter=1000)


In [ ]:
# Load via Model Version Alias
# Reassign alias from the old model version to the new one
client = MlflowClient()

# Set model version alias
model_name = "logistic-regression-model-a"
model_version_alias = "challenger"

model_info = client.get_model_version_by_alias(model_name, model_version_alias)
model_tags = model_info.tags
print(model_tags)

# Get the model version using a model URI
model_uri = f"models:/{model_name}@{model_version_alias}"
model = mlflow.sklearn.load_model(model_uri)

print(model)

## Search Models

In [ ]:
# Searching By Metrics

experimend_id = '254433368725384354'


# Find high-performing models
high_accuracy_models = mlflow.search_logged_models(
    experiment_ids=[experimend_id],  # Replace with your experiment ID
    filter_string="metrics.test_accuracy > 0.9",
)
print(high_accuracy_models)

In [ ]:
from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

## Deploy and Organize Models with Aliases and Tags

In [ ]:
client = MlflowClient()

client.set_registered_model_alias(
    model_name, model_version_alias, "3" #sets "the_best_model_ever" to the version 1 model.
)  # or do it manually in UI

In [ ]:
client = MlflowClient()

client.set_registered_model_alias(
    "creditcard_model_staging", "candidate", "2" #sets "the_best_model_ever" to the version 1 model.
) 

In [ ]:
# get a model version by alias
client.get_model_version_by_alias("creditcard_model_prod", "Champion")

# delete the alias
client.delete_registered_model_alias("creditcard_model_prod", "Sample")

In [ ]:
client = MlflowClient()

ALGORITHM_TYPE = "logistic-regression"
REGISTERED_MODEL_NAME = "fraud-detection-model"

client.set_model_version_tag(
    REGISTERED_MODEL_NAME,
    '9',
    "algorithm",
    ALGORITHM_TYPE
)

In [ ]:
for i in range(22,27):
    client.set_model_version_tag(
        REGISTERED_MODEL_NAME,
        str(i),
        "algorithm",
        ALGORITHM_TYPE
    )

## Deleting a model version from model registry

In [ ]:
# Initialize the MLflow Client
client = MlflowClient()

# Define the name of the registered model and the version to delete
model_name = "fraud-detection-model"
version_to_delete = 21  # Replace with the actual version number

# Delete the specific model version
try:
    client.delete_model_version(name=model_name, version=version_to_delete)
    print(f"Model version {version_to_delete} of '{model_name}' deleted successfully.")
except mlflow.exceptions.MlflowException as e:
    print(f"Error deleting model version: {e}")

## Promoting an MLflow Model across environments

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
client.copy_model_version(
    src_model_uri="models:/creditcard_model_staging@candidate",
    dst_name="creditcard_model_prod",
)

## Renaming an MLflow Model

In [ ]:
client = MlflowClient()
client.rename_registered_model(
    name="logistic-regression-model",
    new_name="logistic-regression-model-d",
)